In [1]:
import pandas as pd
import torch
print(torch.__version__)


2.2.0


In [2]:
df = pd.read_csv('/Users/mac/Downloads/datasets/us-patent/train.csv')
df.describe(include =  'object')

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186


In [3]:
df['input'] = 'TEXT1: '+ df.context + '; TEXT2: ' + df.target + '; ANC: ' + df.anchor
df.input.head()

0    TEXT1: A47; TEXT2: abatement of pollution; ANC...
1    TEXT1: A47; TEXT2: act of abating; ANC: abatement
2    TEXT1: A47; TEXT2: active catalyst; ANC: abate...
3    TEXT1: A47; TEXT2: eliminating process; ANC: a...
4     TEXT1: A47; TEXT2: forest region; ANC: abatement
Name: input, dtype: object

In [4]:
#!pip install datasets
from datasets import Dataset, DatasetDict
ds = Dataset.from_pandas(df)
ds

/opt/anaconda3/envs/pytorch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

In [5]:
#! pip install -U accelerate
#! pip install -U transformers
from transformers import AutoModelForSequenceClassification, AutoTokenizer
model_nm = 'microsoft/deberta-v3-small'
tkz = AutoTokenizer.from_pretrained(model_nm, use_fast = False)

In [6]:
def tok_func(x): return tkz(x['input'])
tok_ds = ds.map(tok_func, batched = True)

Map: 100%|██████████| 36473/36473 [00:07<00:00, 4882.31 examples/s]


In [7]:
row = tok_ds[0]
row['input'],row['input_ids']

('TEXT1: A47; TEXT2: abatement of pollution; ANC: abatement',
 [1,
  54453,
  435,
  294,
  336,
  5753,
  346,
  54453,
  445,
  294,
  47284,
  265,
  6435,
  346,
  23702,
  294,
  47284,
  2])

In [8]:
tok_ds = tok_ds.rename_columns({'score':'labels'})

In [10]:
eval_df = pd.read_csv('/Users/mac/Downloads/datasets/us-patent/test.csv')
eval_df.describe()

,id,anchor,target,context
count,36,36,36,36
unique,36,34,36,29
top,4112d61851461f60,el display,inorganic photoconductor drum,G02
freq,1,2,1,3


In [11]:
dds = tok_ds.train_test_split(.25, seed = 42)
dds

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

In [12]:
eval_df['input'] = 'TEXT1: ' + eval_df.context + '; TEXT2: ' + eval_df.target + '; ANC1: ' + eval_df.anchor
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)

Map: 100%|██████████| 36/36 [00:00<00:00, 562.15 examples/s]


In [13]:
import numpy as np
def corr(x,y): return np.corrcoef(x,y)[0][1]
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

In [14]:
from transformers import TrainingArguments,Trainer
bs = 128
epochs = 4
lr = 8e-5

In [16]:
#!pip install accelerate -U
args = TrainingArguments('outputs', learning_rate = lr, warmup_ratio = 0.1, lr_scheduler_type = 'cosine',
                         evaluation_strategy = 'epoch', per_device_train_batch_size = bs, per_device_eval_batch_size = bs*2, num_train_epochs = epochs,
                         weight_decay = 0.01, report_to = 'none')

In [17]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tkz, compute_metrics=corr_d)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.train()

In [ ]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds = np.clip(preds,0,1)
preds